In [1]:
# 기본 작업 경로 설정

import os
notebook_path = os.path.abspath("../readme.md")
notebook_dir = os.path.dirname(notebook_path)
os.chdir(notebook_dir)

# 현재 작업 디렉토리 출력
print("Current working directory: ", os.getcwd())

Current working directory:  /mnt/e/py_data/project_3_git


### https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md

In [2]:
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# 데이터셋 경로와 형식 설정
register_coco_instances("face_data_set", {}, "data/ssd_rcnn_face/annotations.json", "data/yolo_data/train/")
register_coco_instances("face_data_set_valid", {}, "data/ssd_rcnn_face/annotations_val.json", "data/yolo_data/val/")

# 메타데이터를 확인.
metadata = MetadataCatalog.get("face_data_set")
dataset_dicts = DatasetCatalog.get("face_data_set")

In [3]:
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.data import MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
from detectron2.config import CfgNode as CN
import json
import os

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_dir=None):
        return COCOEvaluator(dataset_name, cfg, True, output_dir)
    
    def train(self):
        self.results = {"train": [], "test": []}
        super().train()
        
        # Add evaluation after each epoch
        for epoch in range(0, self.cfg.SOLVER.MAX_ITER + 1, self.cfg.TEST.EVAL_PERIOD):
            self.do_test(epoch)
            if epoch % self.cfg.SOLVER.CHECKPOINT_PERIOD == 0:
                self.checkpointer.save(f"model_{epoch}")

    def do_test(self, epoch):
        try:
            # Perform evaluation
            evaluator = self.build_evaluator(self.cfg, self.cfg.DATASETS.TEST[0], output_dir=self.cfg.OUTPUT_DIR)
            val_loader = build_detection_test_loader(self.cfg, self.cfg.DATASETS.TEST[0])
            results = inference_on_dataset(self.model, val_loader, evaluator)
            self.results["test"].append({"epoch": epoch, "results": results})
            print("Evaluation results:", results)
    
            # Save the results as JSON
            self.save_results_as_json()
        except Exception as e:
            print(f"Error during evaluation at epoch {epoch}: {e}")
    
    def save_results_as_json(self):
        try:
            output_file_path = os.path.join(self.cfg.OUTPUT_DIR, "train_test_results.json")
            print(f"Saving results to: {output_file_path}")
            with open(output_file_path, 'w') as f:
                json.dump(self.results, f, indent=2)
        except Exception as e:
            print(f"Error saving results as JSON: {e}")


# Set up configuration
cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("face_data_set",)
cfg.DATASETS.TEST = ("face_data_set_valid",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 600000
cfg.INPUT.MIN_SIZE_TRAIN = 512
cfg.INPUT.MAX_SIZE_TRAIN = 512
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.INPUT.MAX_SIZE_TEST = 512
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.INPUT.RANDOM_ROTATION = 30
cfg.INPUT.CROP = CN({"ENABLED": True, "TYPE": "relative_range", "SIZE": [0.8, 0.8]})
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.OUTPUT_DIR = "./models/faster_rcnn_R_50_FPN_3x"
cfg.TEST.EVAL_PERIOD = 1000
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.WARMUP_ITERS = 500

setup_logger()
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[09/06 15:25:53 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[09/06 15:25:54 d2.engine.train_loop]: Starting training from iteration 0


/home/tommy/miniconda3/envs/p3/lib/python3.11/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[09/06 15:26:45 d2.utils.events]:  eta: 0:39:36  iter: 19  total_loss: 2.064  loss_cls: 1.827  loss_box_reg: 0.1404  loss_rpn_cls: 0.08922  loss_rpn_loc: 0.009855    time: 2.4953  last_time: 2.2672  data_time: 1.5805  last_data_time: 1.4263   lr: 1.9962e-05  max_mem: 3888M


2024-09-06 15:26:45.984192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-06 15:26:46.000402: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-06 15:26:46.006009: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-06 15:26:46.019024: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-06 15:26:47.181203: W tensorflow/compiler/tf2

[09/06 15:27:35 d2.utils.events]:  eta: 0:37:49  iter: 39  total_loss: 1.594  loss_cls: 1.298  loss_box_reg: 0.1646  loss_rpn_cls: 0.08414  loss_rpn_loc: 0.009699    time: 2.4312  last_time: 2.6242  data_time: 1.4421  last_data_time: 1.7176   lr: 3.9922e-05  max_mem: 3888M
[09/06 15:28:24 d2.utils.events]:  eta: 0:37:02  iter: 59  total_loss: 0.95  loss_cls: 0.6248  loss_box_reg: 0.2442  loss_rpn_cls: 0.06401  loss_rpn_loc: 0.01028    time: 2.4315  last_time: 1.9801  data_time: 1.5015  last_data_time: 1.1361   lr: 5.9882e-05  max_mem: 3888M
[09/06 15:29:19 d2.utils.events]:  eta: 0:36:54  iter: 79  total_loss: 0.8377  loss_cls: 0.4112  loss_box_reg: 0.379  loss_rpn_cls: 0.03944  loss_rpn_loc: 0.009045    time: 2.5121  last_time: 3.1003  data_time: 1.7906  last_data_time: 2.1011   lr: 7.9842e-05  max_mem: 3888M
[09/06 15:30:11 d2.utils.events]:  eta: 0:36:18  iter: 99  total_loss: 0.7889  loss_cls: 0.3553  loss_box_reg: 0.4028  loss_rpn_cls: 0.02307  loss_rpn_loc: 0.009188    time: 2.53